In [1]:
from transformers import Wav2Vec2Processor, Data2VecAudioModel
import torch
from torch import nn
from datasets import load_dataset
import numpy as np
import librosa.display
import matplotlib.pyplot as plt
import IPython.display as ipd


In [2]:
processor = Wav2Vec2Processor.from_pretrained("facebook/data2vec-audio-base-960h")
encoder = Data2VecAudioModel.from_pretrained("m-a-p/music2vec-v1")


/home/yasser.attia/.conda/envs/SNN/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [3]:
SIZ=30000 #Audio Length

In [ ]:
LI=["VØJ, Narvent - Memory Reboot (4K Music Video)","Üsküdar_a Gider İken","Symphony","Toss a Coin to Your Witcher","Smooth Criminal","The good the bad and the ugly"
    ,"Stronghold - CastleJam","Señorita","Morrison Merrily Kissed The Quaker","Saad Lamjarred - ADDA ELKALAM", "Ahd Al Asdikaa","All I Want - Kodaline","Amr Diab - Qusad Einy","Bella Ciao"
    ,"Fly Me To The Moon","Hamza Namira - Dari Ya Alby","Ladies of the Woods", "Lady Gaga - Shallow","Lady Gaga- Rain On Me","Memories - Maroon 5","Attention","GOT"
    ,"Skrillex - Scary Monsters And Nice Sprites","They_Donot_Care_About_Us","The Misty Mountains Cold - The Hobbit","Fluxxwave","Aria Math (Minecraft) - C418","Mareux - The Perfect Girl" 
    ,"METAMORPHOSIS","Dusk Till Dawn - ZAYN Ft. Sia","Let It Happen - Tame Impala","Interstellar Main Theme - Hans Zimmer","As It Was - Harry Styles","Crystal Castles - KEROSEN","Life in Rio"    
    ,"Daft Punk - Instant Crush","Fine Line - Harry Styles","Numb - Linkin Park","Sparky Deathcap - September","SLOW DANCING IN THE DARK","Faded - Alan Walker","John Legend - All of Me"
    ,"Nour El Ain - Amr Diab","Rockabye","Sia - The Greatest","Sidi Mansour" ,"The Last Of The Mohicans","The Sound Of Silence","Dance Monkey","Desert Rose - Sting & Cheb Mami"
    ,"Despacito","Don_t Stop _Til You Get Enough","Ed Sheeran - Perfect","Ed Sheeran - Shape Of You","HOME - Resonance","Interstellar Theme"
    ,"Adele - Hello","Aicha - Cheb Khaled","Blinding Lights","Charlie Puth - We Don_t Talk Anymore","Cheap Thrills - Sia (Oud cover)","Closer - The Chainsmokers","Coffin Dance",
    "Story"]
dir="/home/yasser.attia/Music_Project/mm/"
sampling_rate=16000
segments1=[]
segments2=[]
#,"Adele - Hello","John Legend - All of Me","Cheap Thrills - Sia (Oud cover)","Closer - The Chainsmokers","Sia - The Greatest",
#    "Faded - Alan Walker","Rockabye","Ed Sheeran - Shape Of You"
summ=0
for pa in LI:
    print(pa)
    pa_ar=dir+pa+"_Arabic"
    pa=dir+pa+".wav"
    pa_ar=pa_ar+".wav"

    y1, sr = librosa.load(pa, sr=sampling_rate)  # Load audio with a specific sampling rate


    y2, sr = librosa.load(pa_ar, sr=sampling_rate)  # Load audio with a specific sampling rate    
    SI1=SIZ*(len(y1)//SIZ)
    SI2=SIZ*(len(y2)//SIZ)
    SI=min(SI1,SI2)
    summ+=(SI//SIZ)
    print(summ)

    #print(y1.mean(),y2.mean())
    audio_files = [y1[i:i+SIZ] for i in range(0, SI-30000, 10000)]  # This splits the audio into chunks

    # Process each audio file to match the model's expected input format
    inputs = [processor(audio, sampling_rate=sampling_rate, return_tensors="pt").input_values for audio in audio_files]
    segments1.extend(inputs)
    audio_files = [y2[i:i+SIZ] for i in range(0, SI-30000, 10000)]  # This splits the audio into chunks

    # Process each audio file to match the model's expected input format
    inputs = [processor(audio, sampling_rate=sampling_rate, return_tensors="pt").input_values for audio in audio_files]
    segments2.extend(inputs)

output_batch = torch.stack(segments2)
input_batch = torch.stack(segments1)

In [5]:
from torch.utils.data import DataLoader, TensorDataset, Subset
import torch
import numpy as np

combined_dataset = TensorDataset(input_batch, output_batch)
total_size = len(combined_dataset)
train_size = int(total_size * 0.8)  # e.g., 80% for training
val_size = total_size - train_size  # the rest for validation
indices = torch.arange(total_size)
t_indices = indices[:train_size]
v_indices = indices[train_size:]
train_subset = Subset(combined_dataset, t_indices)
val_subset = Subset(combined_dataset, v_indices)
train_loader = DataLoader(train_subset, batch_size=256, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=256, shuffle=False)


In [6]:

class Data2VecAudioDecoder(nn.Module):
    def __init__(self, encoder,seq_len):
        super(Data2VecAudioDecoder, self).__init__()
        self.encoder = encoder  # Storing the encoder reference
        self.ratio=int(seq_len/10000)

        # Projecting to a dimension that matches the expected input for the transposed convolutions
        self.initial_projection = nn.Linear(768, 512 )  # Assuming we expand along the sequence length dimension

        # Transposed convolutional layers to gradually upscale the temporal dimension and downscale feature dimension
        self.conv_transpose_layers = nn.ModuleList([
            nn.ConvTranspose1d(512, 512, kernel_size=3, stride=2, padding=1, output_padding=1),  # Increasing length
            nn.ConvTranspose1d(512, 512, kernel_size=3, stride=2, padding=1, output_padding=1),  # Further increasing length
            nn.ConvTranspose1d(512, 256, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ConvTranspose1d(256, 128, kernel_size=5, stride=2, padding=2, output_padding=1),
            nn.ConvTranspose1d(128, 64, kernel_size=5, stride=2, padding=2, output_padding=1),
            nn.ConvTranspose1d(64, 32, kernel_size=5, stride=2, padding=2, output_padding=1),
            nn.ConvTranspose1d(32, 16, kernel_size=5, stride=2, padding=2, output_padding=1),
            nn.ConvTranspose1d(16, 1, kernel_size=5, stride=2, padding=2, output_padding=1)  # Final layer to single channel
        ])

        # Correct LayerNorm configurations assuming output from each conv transpose layer
        self.layer_norms = nn.ModuleList([
            nn.LayerNorm([512, 62*self.ratio]), nn.LayerNorm([512, 124*self.ratio]), nn.LayerNorm([256, 248*self.ratio]),
            nn.LayerNorm([128, 496*self.ratio]), nn.LayerNorm([64, 992*self.ratio]), nn.LayerNorm([32, 1984*self.ratio]),
            nn.LayerNorm([16, 3968*self.ratio]), nn.LayerNorm([1, 7936*self.ratio])  # Final expected size might need adjustment
        ])

        self.activations = nn.ModuleList([nn.GELU() for _ in range(8)])  # Activation functions for each layer
        self.Final_projection = nn.Linear(self.ratio*7936,10000*self.ratio)  # Assuming we expand along the sequence length dimension
        self.flatten = nn.Flatten()

    def forward(self, x):
        # Encoder processing & extracting the last hidden state
        x = self.encoder(x, output_hidden_states=True).last_hidden_state
        x = self.initial_projection(x)

        x = x.view(x.shape[0], x.shape[2], x.shape[1])  # Reshape to match the transposed convolution expectations
        # Sequentially apply transposed convolutions, layer norms, and activations
        for conv, norm, activation in zip(self.conv_transpose_layers, self.layer_norms, self.activations):
            x = conv(x)
            x = norm(x)
            x = activation(x)


        x=self.flatten(x)
        return self.Final_projection(x)


In [7]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Subset
from sklearn.model_selection import KFold

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Data2VecAudioDecoder(encoder, seq_len=SIZ).to(device)  # Model initialization

criterion = nn.MSELoss()


In [9]:
state_dict = torch.load('/home/yasser.attia/Music_Project/path_to_my_model_weights.pth')

model.load_state_dict(state_dict)

<All keys matched successfully>

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
for epoch in range(50):
            train_loss=0
            idx=0
            for orig, cover in train_loader:  # Iterate over the training data
                orig = orig.to(device)
                cover = cover.to(device)
                orig=orig.squeeze(1)
                cover=cover.squeeze(1)

                y_pred = model(orig)
                loss = criterion(y_pred, cover)
                train_loss+=loss.item()
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                idx+=1
            print(f'Epoch: {epoch+1} Training Loss: {train_loss/len(train_loader):.7f}')


            #if epoch%100==999:
                #scheduler.step()
            if epoch%10==0:
                model.eval()
                val_loss=0
                idx=0
                for orig, cover in val_loader:  # Iterate over the training data
                    orig = orig.to(device)
                    cover = cover.to(device)
                    orig=orig.squeeze(1)
                    cover=cover.squeeze(1)

                    y_pred = model(orig)
                    loss = criterion(y_pred, cover)
                    val_loss+=loss.item()
                    idx+=1
                model.train()
                print(f'Epoch: {epoch+1}  Validation Loss: {val_loss/len(val_loader):.7f}')




    

In [ ]:
LI=["VØJ, Narvent - Memory Reboot (4K Music Video)","Üsküdar_a Gider İken","Symphony","Toss a Coin to Your Witcher","Smooth Criminal","The good the bad and the ugly"
    ,"Stronghold - CastleJam","Señorita","Morrison Merrily Kissed The Quaker","Saad Lamjarred - ADDA ELKALAM", "Ahd Al Asdikaa","All I Want - Kodaline","Amr Diab - Qusad Einy","Bella Ciao"
    ,"Fly Me To The Moon","Hamza Namira - Dari Ya Alby","Ladies of the Woods", "Lady Gaga - Shallow","Lady Gaga- Rain On Me","Memories - Maroon 5","Attention","GOT"
    ,"Skrillex - Scary Monsters And Nice Sprites","They_Donot_Care_About_Us","The Misty Mountains Cold - The Hobbit","Fluxxwave","Aria Math (Minecraft) - C418","Mareux - The Perfect Girl" 
    ,"METAMORPHOSIS","Dusk Till Dawn - ZAYN Ft. Sia","Let It Happen - Tame Impala","Interstellar Main Theme - Hans Zimmer","As It Was - Harry Styles","Crystal Castles - KEROSEN","Life in Rio"    
    ,"Daft Punk - Instant Crush","Fine Line - Harry Styles","Numb - Linkin Park","Sparky Deathcap - September","SLOW DANCING IN THE DARK","Faded - Alan Walker","John Legend - All of Me"
    ,"Nour El Ain - Amr Diab","Rockabye","Sia - The Greatest","Sidi Mansour" ,"The Last Of The Mohicans","The Sound Of Silence","Dance Monkey","Desert Rose - Sting & Cheb Mami"
    ,"Despacito","Don_t Stop _Til You Get Enough","Ed Sheeran - Perfect","Ed Sheeran - Shape Of You","HOME - Resonance","Interstellar Theme"
    ,"Adele - Hello","Aicha - Cheb Khaled","Blinding Lights","Charlie Puth - We Don_t Talk Anymore","Cheap Thrills - Sia (Oud cover)","Closer - The Chainsmokers","Coffin Dance",
    "Story"]

In [ ]:
import librosa
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def extract_features(audio_path):
    y, sr = librosa.load(audio_path, sr=None)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=256)
    mfcc_mean = np.mean(mfcc, axis=1)
    return mfcc_mean.reshape(1, -1)

# Paths to your audio files
Summ=0
for song in LI:
    print(song)
    audio_path_ref = "/home/yasser.attia/Music_Project/mm/"+song+"_Arabic.wav"
    audio_path_gen = "/home/yasser.attia/Music_Project/out_dir2/"+song+".wav"

    # Extract features
    features_ref = extract_features(audio_path_ref)
    features_gen = extract_features(audio_path_gen)

    # Compute cosine similarity
    similarity = cosine_similarity(features_ref, features_gen)
    Summ+=similarity[0][0]
    print(f'Cosine similarity: {similarity[0][0]}')



In [ ]:
print("Average similarty: ", Summ/len(LI))

In [ ]:
for song in LI:
        LII=[ song]

        import soundfile as sf

        dir="/home/yasser.attia/Music_Project/mm/"
        sampling_rate=16000

        output_dir = "/home/yasser.attia/Music_Project/out_dir3/" 
        for pa in LII:
                file_inp=dir+pa+".wav"
                output_path=output_dir+pa+".wav"
                ar_files=dir+pa+"_Arabic.wav"
                print(pa)

                y1, sr = librosa.load(file_inp, sr=sampling_rate)  # Load audio with a specific sampling rate
                y2, sr2 = librosa.load(ar_files, sr=sampling_rate)  # Load audio with a specific sampling rate

                audio_files = [y1[i:i+SIZ] for i in range(0, len(y1)-30000, 30000)]  # This splits the audio into chunks
                inputs = [processor(audio, sampling_rate=sampling_rate, return_tensors="pt").input_values for audio in audio_files]
                input_batch = torch.stack(inputs).to(device)
                input_batch=input_batch.squeeze(1)
                y=[]
                for i in input_batch:
                        j=model(i.unsqueeze(0)) 
                        y.extend(j.cpu().detach().numpy().tolist()[0])
        y=np.array(y)

        D = librosa.stft(y2, n_fft=n_fft, hop_length=hop_length)
        S_db = librosa.amplitude_to_db(abs(D), ref=np.max)

        plt.figure(figsize=(10, 4))
        librosa.display.specshow(S_db, sr=sr, hop_length=hop_length, x_axis='time', y_axis='hz')
        plt.colorbar(format='%+2.0f dB')
        plt.title('Spectrogram for '+pa+' oud couver Ground-truth')
        plt.savefig('/home/yasser.attia/Music_Project/spectogram/'+pa+'_GT.png', dpi=300)  # Save as PNG with high resolution

        plt.show()

        n_fft = 2048  # Higher frame size
        hop_length = 512  # Smaller hop length
        D = librosa.stft(y1, n_fft=n_fft, hop_length=hop_length)
        S_db = librosa.amplitude_to_db(abs(D), ref=np.max)

        plt.figure(figsize=(10, 4))
        librosa.display.specshow(S_db, sr=sr, hop_length=hop_length, x_axis='time', y_axis='hz')
        plt.colorbar(format='%+2.0f dB')
        plt.title('Spectrogram for original music of '+pa)
        plt.savefig('/home/yasser.attia/Music_Project/spectogram/'+pa+'_inp.png', dpi=300)  # Save as PNG with high resolution

        plt.show()

        n_fft = 2048  # Higher frame size
        hop_length = 512  # Smaller hop length
        D = librosa.stft(y, n_fft=n_fft, hop_length=hop_length)
        S_db = librosa.amplitude_to_db(abs(D), ref=np.max)

        plt.figure(figsize=(10, 4))
        librosa.display.specshow(S_db, sr=sr, hop_length=hop_length, x_axis='time', y_axis='hz')
        plt.colorbar(format='%+2.0f dB')
        plt.title('Spectrogram for Generated '+pa+' oud Cover')
        plt.savefig('/home/yasser.attia/Music_Project/spectogram/'+pa+'_oud.png', dpi=300)  # Save as PNG with high resolution

        plt.show()

In [ ]:
import soundfile as sf

dir="/home/yasser.attia/Music_Project/mm/"
sampling_rate=16000

output_dir = "/home/yasser.attia/Music_Project/out_dir3/" 
for pa in LI:
    file_inp=dir+pa+".wav"
    output_path=output_dir+pa+".wav"
    print(pa)

    y1, sr = librosa.load(file_inp, sr=sampling_rate)  # Load audio with a specific sampling rate
    audio_files = [y1[i:i+SIZ] for i in range(0, len(y1)-30000, 30000)]  # This splits the audio into chunks
    inputs = [processor(audio, sampling_rate=sampling_rate, return_tensors="pt").input_values for audio in audio_files]
    input_batch = torch.stack(inputs).to(device)
    input_batch=input_batch.squeeze(1)
    y=[]
    for i in input_batch:
            j=model(i.unsqueeze(0)) 
            y.extend(j.cpu().detach().numpy().tolist()[0])
    sf.write(output_path, y, samplerate=sampling_rate)
    print(f"Saved: {output_path}")
